<a href="https://colab.research.google.com/github/PersGraphics/CNN/blob/main/4_2_PARTE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# Montamos el Google Drive en el directorio del proyecto y
# descomprimios el fichero con los datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!unzip -n '/content/drive/MyDrive/DL_flowers.zip' >> /dev/null

In [ ]:
# Especificamos los paths al directorio que contiene
# las imagenes y al fichero con las etiquetas
data_path = 'flowers/'
imgs_path = data_path + "images/"
labels_path = data_path + "labels.csv"


In [ ]:
# Leemos el fichero CSV con las etiquetas
labels = pd.read_csv(labels_path, dtype = {"class": "category"})

In [ ]:
# Creamos las tres particiones de datos: entrenamiento, validación y test
seed = 0
train_data, test_data = train_test_split(labels, test_size=0.3, random_state=seed)
dev_data, test_data = train_test_split (test_data, test_size=0.5, random_state=seed)
# Actualizamos los indices de cada particion
train_data = train_data.reset_index(drop=True)
dev_data = dev_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [ ]:
# Preparamos lo dato utilizando normalización (rescale)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)# Especificamos una serie de parámetros dependientes del conjunto de datos
classes = ["0", "1","2","3","4"]# clases del problema
n_classes = len(classes)# numero de clases
img_width = img_height = 224 # dimensiones de la imagen
x_col = 'image_name'# nombres de las columnas en el fichero CSV
y_col = 'class'

In [ ]:
# Generamos los batches con los datos para las tres particiones
batch_size = 128# tamaño del batch
train_generator = datagen.flow_from_dataframe(dataframe=train_data, directory=imgs_path, x_col=x_col, y_col=y_col,has_ext=True, class_mode="categorical", target_size=(img_width, img_height),batch_size=batch_size, classes=classes)
dev_generator = datagen.flow_from_dataframe(dataframe=dev_data, directory=imgs_path, x_col=x_col, y_col=y_col,has_ext=True, class_mode="categorical", target_size=(img_width,img_height),batch_size=batch_size, classes=classes)
test_generator = datagen.flow_from_dataframe(dataframe=test_data, directory=imgs_path, x_col=x_col, y_col=y_col,has_ext=True, class_mode="categorical", target_size=(img_width, img_height),batch_size=batch_size, classes=classes)

Found 3020 validated image filenames belonging to 5 classes.
Found 647 validated image filenames belonging to 5 classes.
Found 648 validated image filenames belonging to 5 classes.


In [ ]:
# Cargamos el modelo InceptionV3 pre-entrenado con ImageNet
base_model = tf.keras.applications.InceptionV3 (weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
# En este ejemplo vamos a ajustar los parámetros de las nuevas capas del model
# dejando fijos los parámetros del resto de capa
# (es decir, que no se actualicen durante el entrenamiento)
for layer in base_model.layers:
  layer.trainable = False
# por defecto, el valor de trainable es True

In [ ]:
# Añadimos nuevas capas al final para adaptar el modelo a nuestro problema
x = base_model.output
x = tf.keras. layers. GlobalAveragePooling2D ()(x)
x = tf. keras. layers. Dense (1024, activation='relu') (x)
# Añadimos una última capa completamente conectada con 5 neuronas
# (número de clases) para obtener la salida de la red
predictions = tf.keras. layers.Dense(n_classes, activation='softmax') (x)

In [ ]:
# Creamos el modelo final y lo compilamos
model = tf.keras.models.Model(inputs= [base_model.input], outputs=[predictions])
model.summary()
# representación en modo texto del modelo
model. compile(
  loss='categorical_crossentropy', # función de pérdida para problemas de clasificación multi-clase
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  # optimizador Adam, learning rate (1r)
  metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
# Entrenamos el modelo con los datos preparados
model.fit(
  train_generator,
  epochs=10,
  # numero de epochs
  verbose=2,
  # muestra informacion del error al finalizar cada epoch
  steps_per_epoch=len(train_data)/batch_size,
  validation_data=dev_generator,
  validation_steps=len(dev_data)/batch_size,
  use_multiprocessing = True # Para que vaya mas rapido
)

Epoch 1/10
23/23 - 57s - loss: 1.5232 - accuracy: 0.3778 - val_loss: 1.2958 - val_accuracy: 0.5131 - 57s/epoch - 2s/step
Epoch 2/10
23/23 - 49s - loss: 1.1687 - accuracy: 0.5881 - val_loss: 1.0544 - val_accuracy: 0.6584 - 49s/epoch - 2s/step
Epoch 3/10
23/23 - 46s - loss: 0.9628 - accuracy: 0.6914 - val_loss: 0.9129 - val_accuracy: 0.7002 - 46s/epoch - 2s/step
Epoch 4/10
23/23 - 42s - loss: 0.8389 - accuracy: 0.7368 - val_loss: 0.8263 - val_accuracy: 0.7357 - 42s/epoch - 2s/step
Epoch 5/10
23/23 - 49s - loss: 0.7513 - accuracy: 0.7662 - val_loss: 0.7548 - val_accuracy: 0.7666 - 49s/epoch - 2s/step
Epoch 6/10
23/23 - 49s - loss: 0.6878 - accuracy: 0.7887 - val_loss: 0.7073 - val_accuracy: 0.7774 - 49s/epoch - 2s/step
Epoch 7/10
23/23 - 48s - loss: 0.6392 - accuracy: 0.8036 - val_loss: 0.6695 - val_accuracy: 0.7867 - 48s/epoch - 2s/step
Epoch 8/10
23/23 - 49s - loss: 0.5996 - accuracy: 0.8166 - val_loss: 0.6360 - val_accuracy: 0.8053 - 49s/epoch - 2s/step
Epoch 9/10
23/23 - 44s - loss: 0

In [ ]:
# Por ultimo, podemos evaluar el modelo en el conjunto de test
test_loss, test_acc = model. evaluate(
  test_generator,
  steps=len (test_data)/batch_size,
  verbose=1)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_acc))

5/5 [==============================] - 5s 934ms/step - loss: 0.5738 - accuracy: 0.8117
test_loss: 0.5738, test_acc: 0.8117
